In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from bs4 import BeautifulSoup
import os

In [ ]:
ruta_archivo = "/kaggle/input/html2-1/unstuffed-cabbage-roll.html"

In [ ]:
# Usamos 'utf-8' para evitar problemas con tildes o caracteres especiales
with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
    contenido_html = archivo.read()

In [ ]:
soup = BeautifulSoup(contenido_html, 'html.parser')

In [ ]:
# Extracting the recipe title
title = soup.find("title")
title.string

In [ ]:
#score
#description
#prep time
#cooking time
#total time
#servings
#ingredients
#directtions
#nutritions facts
#Reviews

In [ ]:
score=soup.find("div", {"id":"mm-recipes-review-bar__rating_1-0"})
score.string

In [ ]:
description=soup.find("p", {"class":"article-subheading text-utility-300"})
description.string

In [ ]:
prep_time=soup.find("div", {"class":"mm-recipes-details__value"})
prep_time.string

In [ ]:
def parsearReceta(html_doc_path):
    with open(html_doc_path, 'r', encoding='utf-8') as archivo:
        html_doc = archivo.read()

    soup = BeautifulSoup(html_doc, 'html.parser')
    
    score=soup.find("div", {"id":"mm-recipes-review-bar__rating_1-0"}).string
    prep_time=soup.find("div", {"class":"mm-recipes-details__value"}).string
    description=soup.find("p", {"class":"article-subheading text-utility-300"}).string
    title = soup.find("title").string
    return {"score":score,"prep_time":prep_time, "description":description, "title":title}

In [ ]:
htmlparseado = parsearReceta(contenido_html)

htmlparseado

In [ ]:
ruta_receta2 = "/kaggle/input/receta-2/chicken-cobbler-recipe-7966464.html"

In [ ]:
receta2 = parsearReceta(ruta_receta2)
receta2